# RESNET

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
import glob
from PIL import Image
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Input, ZeroPadding2D, BatchNormalization, Activation, AveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense

Using TensorFlow backend.


In [3]:
train_datagen=ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
training_set=train_datagen.flow_from_directory('dataset/train',target_size=(256,256),batch_size=32,class_mode='categorical')

Found 2424 images belonging to 5 classes.


In [4]:
test_datagen=ImageDataGenerator(rescale=1./255) 
test_set=test_datagen.flow_from_directory('dataset/Test',target_size=(256,256),batch_size=32,class_mode='categorical')

Found 735 images belonging to 5 classes.


In [5]:
def identity_block(X, f, filters):
    # Retrieve Filters
    F1, F2, F3 = filters

    # First component of main path
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid')(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)

    # Second component of main path
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same')(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)

    # Third component of main path
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid')(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)

    return X

In [6]:
def convolutional_block(X, f, filters, s): 
    # Retrieve Filters
    F1, F2, F3 = filters

    # First component of main path 
    X = Conv2D(F1, (1, 1), strides = (s,s))(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)

    # Second component of main path
    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same')(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)

    # Third component of main path
    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid')(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X) 
    
    return X

In [7]:
def ResNet50(input_shape, classes):   
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)
    
    # Stage 1
    X = Conv2D(64, (7, 7), strides = (2, 2))(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f = 3, filters = [64, 64, 256], s = 1)
    X = identity_block(X, 3, [64, 64, 256])
    X = identity_block(X, 3, [64, 64, 256])

    # Stage 3
    X = convolutional_block(X, f = 3, filters = [128, 128, 512], s = 3)
    X = identity_block(X, 3, [128, 128, 512])
    X = identity_block(X, 3, [128, 128, 512])
    X = identity_block(X, 3, [128, 128, 512])

    # Stage 4
    X = convolutional_block(X, f = 3, filters = [256, 256, 1024], s = 2)
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])

    # Stage 5
    X = convolutional_block(X, f = 3, filters = [512, 512, 2048], s = 2)
    X = identity_block(X, 3, [512, 512, 2048])
    X = identity_block(X, 3, [512, 512, 2048])

    # AVGPOOL.
    X = AveragePooling2D((2, 2))(X)

    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='sigmoid')(X)
    
    # Create model
    model = Model(inputs = X_input, outputs = X)

    return model

In [8]:
model = ResNet50(input_shape = (256, 256, 3), classes = 5)
model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
zero_padding2d (ZeroPadding2 (None, 262, 262, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 128, 128, 64)      9472      
_________________________________________________________________
batch_normalization (BatchNo (None, 128, 128, 64)      256       
_________________________________________________________________
activation (Activation)      (None, 128, 128, 64)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 63, 63, 64)        0         
_________

In [9]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
mc = ModelCheckpoint('resnet.h5', monitor = 'accuracy', verbose=1, save_best_only = True)

In [ ]:
batch_size = 200
epochs = 10

#### Fitting the model
history = model.fit_generator(
           training_set, steps_per_epoch=training_set.samples // batch_size, 
           epochs=epochs, 
           validation_data=test_set,validation_steps=test_set.samples // batch_size,
           callbacks=[mc])

Epoch 1/10
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
37/37 [==============================] - 1263s 34s/step - loss: 1.8973 - acc: 0.2100 - val_loss: 1.6049 - val_acc: 0.1960
Epoch 2/10
37/37 [==============================] - 1256s 34s/step - loss: 1.7443 - acc: 0.2171 - val_loss: 1.6075 - val_acc: 0.2131
Epoch 3/10
37/37 [==============================] - 1259s 34s/step - loss: 1.6448 - acc: 0.2154 - val_loss: 1.6038 - val_acc: 0.2273
Epoch 4/10
37/37 [==============================] - 1248s 34s/step - loss: 1.6077 - acc: 0.2066 - val_loss: 1.6038 - val_acc: 0.2273
Epoch 5/10
37/37 [==============================] - 1261s 34s/step - loss: 1.6083 - acc: 0.2162 - val_loss: 1.6057 - val_acc: 0.2273
Epoch 6/10
37/37 [==============================] - 1320s 36s/step - loss: 1.6071 - acc: 0.2083 - val_loss: 1.6043 - val_acc: 0.1960
Epoch 7/10
37/37 [==============================] - 1291s 35s/step - loss: 1.6961 - acc: 0.2061 - val_loss: 

In [ ]:
def graph():
    #Plot training & validation accuracy values
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()

    # Plot training & validation loss values
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()

In [ ]:
graph()